In [8]:
import os
import numpy as np
from plio.examples import get_path

# Reads a .atf file and outputs all of the 
# .ipf, .gpf, .sup, .prj, and path to locate the 
# .apf file (should be the same as all others) 
def read_atf(atf_file):
    with open(get_path(atf_file)) as f:
        
        atf_file = get_path(atf_file);
        
        files = []
        ipf = []
        sup = []
        files_dict = []
        
        # Grabs every PRJ, GPF, SUP, and IPF image from the ATF file
        for line in f:
            if line[-4:-1] == 'prj' or line[-4:-1] == 'gpf' or line[-4:-1] == 'sup' or line[-4:-1] == 'ipf' or line[-4:-1] == 'atf':
                files.append(line)
        
        files = np.array(files)
        
        # Creates appropriate arrays for certain files in the right format
        for file in files:
            file = file.strip()
            file = file.split(' ')

            # Grabs all the IPF files
            if file[1].endswith('.ipf'):
                ipf.append(file[1])

            # Grabs all the SUP files
            if file[1].endswith('.sup'):
                sup.append(file[1])

            files_dict.append(file)

        # Creates a dict out of file lists for GPF, PRJ, IPF, and ATF
        files_dict = (dict(files_dict))
        
        # Sets the value of IMAGE_IPF to all IPF images
        files_dict['IMAGE_IPF'] = ipf
        
        # Sets the value of IMAGE_SUP to all SUP images
        files_dict['IMAGE_SUP'] = sup
        
        # Sets the value of PATH to the path of the ATF file
        files_dict['PATH'] = os.path.dirname(os.path.abspath(atf_file))
        
        return files_dict
    
read_atf('CTX_Athabasca_Middle_step0.atf')

{'ATF_FILE': 'CTX_Athabasca_Middle_step0.atf',
 'GP_FILE': 'CTX_Athabasca_Middle.gpf',
 'IMAGE_IPF': ['P19_008344_1894_XN_09N203W.ipf',
  'P20_008845_1894_XN_09N203W.ipf',
  'P03_002371_1888_XI_08N204W.ipf',
  'P01_001540_1889_XI_08N204W.ipf',
  'P01_001606_1897_XI_09N203W.ipf',
  'P03_002226_1895_XI_09N203W.ipf'],
 'IMAGE_SUP': ['P19_008344_1894_XN_09N203W.sup',
  'P20_008845_1894_XN_09N203W.sup',
  'P03_002371_1888_XI_08N204W.sup',
  'P01_001540_1889_XI_08N204W.sup',
  'P01_001606_1897_XI_09N203W.sup',
  'P03_002226_1895_XI_09N203W.sup'],
 'PATH': '/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/SocetSet',
 'PROJECT': 'D:\\data\\CTX_Athabasca_Middle.prj'}

In [57]:
import os
import sys
import math

sys.path.insert(0, os.path.abspath('/home/tthatcher/Desktop/Projects/Plio/plio'))

import pandas as pd
import numpy as np

from plio.examples import get_path
from plio.io.io_bae import read_gpf
from plio.io.io_gdal import GeoDataset


In [11]:
def read_ipf(input_data):
    """
    Read a socet ipf file into a pandas data frame

    Parameters
    ----------
    input_data : str
                 path to the an input data file

    Returns
    -------
    df : pd.DataFrame
         containing the ipf data with appropriate column names and indices
    """

    # Check that the number of rows is matching the expected number
    with open(input_data, 'r') as f:
        for i, l in enumerate(f):
            if i == 1:
                cnt = int(l)
            elif i == 2:
                col = l
                break

    # TODO: Add unicode conversion

    d = [line.split() for line in open(input_data, 'r')]
    d = np.hstack(np.array(d[3:]))
    d = d.reshape(-1, 12)

    assert int(cnt) == len(d), 'Dataframe length {} does not match point length {}.'.format(int(cnt), len(df))

    return d

def read_ipfs(input_data_list):
    """
    Read a socet ipf file into a pandas data frame

    Parameters
    ----------
    input_data_list : list
                 list of paths to the a set of input data files

    Returns
    -------
    df : pd.DataFrame
         containing the ipf data with appropriate column names and indices
    """

    columns = np.genfromtxt(input_data_list[0], skip_header=2, dtype='unicode',
                                    max_rows = 1, delimiter = ',')

    d_total = []

    for input_file in input_data_list:
        d = read_ipf(input_file)
        for point in d:
            d_total.append(point)

    df = pd.DataFrame(d_total, columns=columns)

    # Soft conversion of numeric types to numerics, allows str in first col for point_id
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

In [12]:
gpf_file = get_path('CTX_Athabasca_Middle.gpf');

ipf_list = [get_path('P20_008845_1894_XN_09N203W.ipf'),
            get_path('P03_002371_1888_XI_08N204W.ipf'),
            get_path('P01_001540_1889_XI_08N204W.ipf'),
            get_path('P01_001606_1897_XI_09N203W.ipf'),
            get_path('P03_002226_1895_XI_09N203W.ipf'),]

# ipf_list = [get_path('P20_008845_1894_XN_09N203W.ipf')]

print(ipf_list)
gpf_df = read_gpf(gpf_file).set_index('point_id')
ipf_df = read_ipfs(ipf_list).set_index('pt_id')
# ipf_df.to_csv("/Users/adampaquette/repos/plio/plio/examples/SocetSet/P20_008845_1894_XN_09N203W.csv")

['/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/SocetSet/P20_008845_1894_XN_09N203W.ipf', '/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/SocetSet/P03_002371_1888_XI_08N204W.ipf', '/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/SocetSet/P01_001540_1889_XI_08N204W.ipf', '/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/SocetSet/P01_001606_1897_XI_09N203W.ipf', '/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/SocetSet/P03_002226_1895_XI_09N203W.ipf']


In [41]:
new_df = ipf_df.merge(gpf_df, left_index=True, right_index=True)
# print(new_df.columns)
new_df.index('')

,val,fid_val,no_obs,l.,s.,sig_l,sig_s,res_l,res_s,fid_x,...,known,lat_Y_North,long_X_East,ht,sigma0,sigma1,sigma2,res0,res1,res2
10_8344_8845_4r,1,0,0,-4044.697510,1008.950928,0.000000,0.000000,0.063678,0.661294,0.0,...,0,0.159378,2.724649,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,1700.584473,-2390.001709,0.000000,0.000000,-0.063695,-1.055619,0.0,...,0,0.159378,2.724649,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
10_8344_8845_4r,1,0,0,2006.141113,-2234.915283,0.000000,0.000000,0.318779,-0.585138,0.0,...,0,0.159378,2.724649,-2523.828227,0.0,0.0,25.000000,18.301328,44.206259,416.201741
11_8344_8845_4r,1,0,0,-761.216064,2303.787109,0.000000,0.000000,-0.247307,-0.587299,0.0,...,0,0.164905,2.721815,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,4985.437988,-1070.364990,0.000000,0.000000,0.239207,1.080801,0.0,...,0,0.164905,2.721815,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
11_8344_8845_4r,1,0,0,5293.700195,-993.390625,0.000000,0.000000,0.672877,0.626316,0.0,...,0,0.164905,2.721815,-2445.237027,0.0,0.0,30.000000,-22.046575,103.403228,173.210013
12_8344_8845_4r,1,0,0,-889.364441,966.533997,0.000000,0.000000,-0.251753,0.008170,0.0,...,0,0.164949,2.724076,-2606.935163,0.0,0.0,100.000000,-7.549561,93.170584,180.058858
12_8344_8845_4r,1,0,0,4856.525391,-2439.154785,0.000000,0.000000,0.133406,0.068024,0.0,...,0,0.164949,2.724076,-2606.935163,0.0,0.0,100.000000,-7.549561,93.170584,180.058858
12_8344_8845_4r,1,0,0,5158.205566,-2295.737549,0.000000,0.000000,0.487209,0.036399,0.0,...,0,0.164949,2.724076,-2606.935163,0.0,0.0,100.000000,-7.549561,93.170584,180.058858
13_8344_8845_4r,1,0,0,-2559.871338,1777.522827,0.000000,0.000000,-0.131265,-0.305102,0.0,...,0,0.161840,2.723059,-2551.901554,0.0,0.0,4.536068,-1.643694,74.244153,319.554535


In [52]:
files_dict = read_atf('CTX_Athabasca_Middle_step0.atf')['IMAGE_SUP']

lines_samples_dict = {}
for key in files_dict:
    with open(key) as f:
        if f[0:5] == 'LINES':
            lines_samples_dict[]
        if f[0:7] == 'SAMPLES':
            lines_samples_dict
print(new_df.index[800])
sup_dict = {}
for index in new_df['l.']:
    if index in files_dict:
            new_df['l.'] = 

print(sup_dict)
        
        atf_file = get_path(atf_file);
        
        files = []
        ipf = []
        sup = []
        files_dict = []
        
        # Grabs every PRJ, GPF, SUP, and IPF image from the ATF file
        for line in f:
            if line[-4:-1] == 'prj' or line[-4:-1] == 'gpf' or line[-4:-1] == 'sup' or line[-4:-1] == 'ipf' or line[-4:-1] == 'atf':
                files.append(line)
        
        files = np.array(files)

P20_008845_1894_XN_09N203W_7


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
diff = ipf_df.index.difference(pd.unique(gpf_df.index))

total_similar = 0
for i in gpf_df.index:
    for j in pd.unique(ipf_df.index):
        if i == j:
            total_similar += 1
diff

In [61]:

# double ToPlanetocentric3(const double lattitude, double eRadius, double pRadius)
# {

#    double mylat = (lattitude-90.0)*DegToRad;
#    if(fabs(mylat)<90.0)
#    {
#       mylat = atan((tan(mylat) * (pRadius/eRadius) * (pRadius/eRadius)) );
#    }
#    return (mylat*RadToDeg)+90.0;
# }



# double ToPlanetocentric4(const double lattitude, double eRadius, double pRadius)
# {

#    double mylat = (lattitude-90.0)*DegToRad;
#    if(fabs(mylat)<90.0)
#    {
#       double oe = acos(pRadius/eRadius);
#       double csqrd = cos(oe)*cos(oe);
#       mylat = atan(csqrd*tan(mylat));
#    }
#    return (mylat*RadToDeg)+90.0;
# }


test = GeoDataset('/home/tthatcher/Desktop/Projects/Plio/plio/plio/examples/Apollo15/AS15-M-0295_SML_geo.tif')

print(test.spatial_reference)
print(test.geospatial_coordinate_system)
def ToPlanetocentric3(lat, eRadius, pRadius):
    mylat = math.radians(lat - 90.0)
    
    if(math.fabs(mylat) < 90.0):
        mylat = math.atan(math.tan(mylat) * (pRadius/eRadius) * (pRadius/eRadius))
    
    return math.degrees(mylat) + 90.0

def ToPlanetocentric4(lat, eRadius, pRadius):
    mylat = math.radians(lat - 90.0)
    
    if(math.fabs(mylat) < 90.0):
        oe = math.acos(pRadius/eRadius)
        cos_squared = math.cos(oe)**2
        mylat = math.atan(cos_squared*(math.tan(mylat)))
    
    return math.degrees(mylat) + 90.0

GEOGCS["unnamed ellipse",
    DATUM["unknown",
        SPHEROID["Unknown",1737400,0]],
    PRIMEM["Greenwich",0],
    UNIT["Degree",0.017453292519943295]]
GEOGCS["unnamed ellipse",
    DATUM["unknown",
        SPHEROID["Unknown",1737400,0]],
    PRIMEM["Greenwich",0],
    UNIT["Degree",0.017453292519943295]]
